In [48]:
#Importamos las librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import Reader,Dataset
from surprise.model_selection import train_test_split, KFold, RandomizedSearchCV
from surprise.prediction_algorithms.knns import KNNWithMeans
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import accuracy

In [2]:
#Importamos todo el dataset de reviews completo
for i in range(1,10):
    if i == 1:
        reviews = pd.read_csv('D:/Marcos/HENRY/Proyecto-Final/reviews_finales/reviews_1.csv',sep=';',escapechar='\\')
    else:
        reviews = pd.concat([reviews,pd.read_csv('D:/Marcos/HENRY/Proyecto-Final/reviews_finales/reviews_'+str(i)+'.csv',
                                                 sep=';',escapechar='\\')],ignore_index=True)
reviews.head()

C:\Users\larra\AppData\Local\Temp\ipykernel_7188\1376262560.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews = pd.concat([reviews,pd.read_csv('D:/Marcos/HENRY/Proyecto-Final/reviews_finales/reviews_'+str(i)+'.csv',


,Id_Usuario,Id_Restaurant,Rating,Reseña,Timestamp
0,117975922625527873536.0,0x8889221157fb3455:0x5c125c40c3eccc2a,4,"On the higher end of price for pizza, but they...",1463443013514
1,114316524790614294528.0,0x8889221157fb3455:0x5c125c40c3eccc2a,3,"Food was ok, felt like the atmosphere as well ...",1447623939865
2,113632573920508362752.0,0x8889221157fb3455:0x5c125c40c3eccc2a,4,"Good food, service so so",1469293549247
3,107684037632376815616.0,0x8889221157fb3455:0x5c125c40c3eccc2a,5,Love it,1442279219480
4,112603593315264724992.0,0x8889221157fb3455:0x5c125c40c3eccc2a,5,Yum,1382634896130


In [3]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16007867 entries, 0 to 16007866
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   Id_Usuario     object
 1   Id_Restaurant  object
 2   Rating         int64 
 3   Reseña         object
 4   Timestamp      int64 
dtypes: int64(2), object(3)
memory usage: 610.7+ MB


In [4]:
#Importamos el dataset de restaurants
restaurants = pd.read_csv('D:/Marcos/HENRY/Proyecto-Final/restaurants_homolog.csv')
restaurants.head()

C:\Users\larra\AppData\Local\Temp\ipykernel_7188\3270671378.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  restaurants = pd.read_csv('D:/Marcos/HENRY/Proyecto-Final/restaurants_homolog.csv')


,Unnamed: 0,Id_Restaurant,Nombre,Ciudad,Estado,Cod_postal,Latitud,Longitud,Tipo,Ambiente,...,Tipos de pago,"Opciones (Des,Com,Cena)",Tipo director,Vestimenta,Salud y Seguridad,Rango de precios,Aspectos destacados,Fumar,Rating_promedio,Cant_reviews
0,0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,Los Angeles,CA,90005,34.058092,-118.292130,Comida Oriental,1,...,0,0,0,0,0,0,0,0,4.428571,14
1,1,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,La Mirada,CA,90638,33.916402,-118.010855,Comida Rápida,1,...,0,1,0,0,1,0,0,0,4.833333,6
2,2,0x87ec235c54d25b31:0x3b75fb5facc602f,"Sweet Rewards Gluten Free Bakery, LLC",Waukee,IA,50263,41.616079,-93.865487,Saludable/Vegano,0,...,0,0,0,0,0,0,0,0,5.000000,9
3,3,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,Austin,TX,78702,30.273985,-97.719564,Otros,1,...,0,1,0,0,0,0,0,0,4.125000,8
4,4,0x4cb29c1dea53f29f:0xc606af14a0995094,Benny's Famous Fried Clams,Portland,ME,4101,43.643366,-70.272457,Mariscos,0,...,0,0,0,0,0,0,0,0,2.833333,6


In [5]:
restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171419 entries, 0 to 171418
Data columns (total 32 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               171419 non-null  int64  
 1   Id_Restaurant            171419 non-null  object 
 2   Nombre                   171419 non-null  object 
 3   Ciudad                   171419 non-null  object 
 4   Estado                   171419 non-null  object 
 5   Cod_postal               171398 non-null  object 
 6   Latitud                  171419 non-null  float64
 7   Longitud                 171419 non-null  float64
 8   Tipo                     171419 non-null  object 
 9   Ambiente                 171419 non-null  int64  
 10  Para grupos              171419 non-null  int64  
 11  Promociones              171419 non-null  int64  
 12  Estacionamiento          171419 non-null  int64  
 13  Mejores noches           171419 non-null  int64  
 14  Incl

Para realizar el modelo, vamos a filtrar por los restaurants que pertenezcan al estado de New York. Esto evitará el alto cosoto computacional de tuning y entrenamiento del modelo. Siempre puede extenderse el mismo proceso a todo EE.UU.


In [6]:
#Filtramos los retaurants que sean de New York
restaurants_ny = restaurants[restaurants['Estado']=='NY']
restaurants_ny.head()

,Unnamed: 0,Id_Restaurant,Nombre,Ciudad,Estado,Cod_postal,Latitud,Longitud,Tipo,Ambiente,...,Tipos de pago,"Opciones (Des,Com,Cena)",Tipo director,Vestimenta,Salud y Seguridad,Rango de precios,Aspectos destacados,Fumar,Rating_promedio,Cant_reviews
16,16,0x89c261f60bdf13db:0x38da730e4687a97b,Dunkin',Queens,NY,11361,40.763985,-73.771430,Comida Rápida,1,...,1,1,0,0,0,0,1,0,3.166667,6
18,18,0x89c258ffaeaba947:0x8355860772a595a9,Raffaello Kosher Pizza,New York,NY,10036,40.756872,-73.980427,Comida Italiana,1,...,0,1,0,0,0,0,0,0,3.400000,5
22,22,0x89c244427d3e2c19:0xc1dbf7d8e71d7201,Casa Malinche,Brooklyn,NY,11235,40.581086,-73.960139,Comida Latinoamericana,0,...,0,0,0,0,0,0,0,0,4.600000,5
32,32,0x89c2f5c3caef3fb5:0xb7f855503b4bc974,Kennedy's Chicken & Sandwiches,Bronx,NY,10454,40.808010,-73.919617,Comida Americana,1,...,0,1,0,0,0,0,1,0,4.166667,6
46,46,0x89c258b8e75311b5:0x745721b4d1be3147,V-Note,New York,NY,10075,40.772618,-73.952116,Comida Italiana,1,...,0,1,0,0,0,0,1,0,4.272727,11


In [7]:
restaurants_ny.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12063 entries, 16 to 123556
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               12063 non-null  int64  
 1   Id_Restaurant            12063 non-null  object 
 2   Nombre                   12063 non-null  object 
 3   Ciudad                   12063 non-null  object 
 4   Estado                   12063 non-null  object 
 5   Cod_postal               12063 non-null  object 
 6   Latitud                  12063 non-null  float64
 7   Longitud                 12063 non-null  float64
 8   Tipo                     12063 non-null  object 
 9   Ambiente                 12063 non-null  int64  
 10  Para grupos              12063 non-null  int64  
 11  Promociones              12063 non-null  int64  
 12  Estacionamiento          12063 non-null  int64  
 13  Mejores noches           12063 non-null  int64  
 14  Inclusivo           

In [8]:
#Son 12.063 lugares. Filtraremos estos en el dataset de reviews
reviews_ny = reviews[reviews['Id_Restaurant'].isin(restaurants_ny['Id_Restaurant'].values)]
reviews_ny.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 743079 entries, 6360731 to 7484519
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Id_Usuario     743079 non-null  object
 1   Id_Restaurant  743079 non-null  object
 2   Rating         743079 non-null  int64 
 3   Reseña         427330 non-null  object
 4   Timestamp      743079 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 34.0+ MB


In [9]:
#Son 743.079 reviews.
#Vamos a filtrarlas aun mas puesto que para los usuarios que hicieron una sola reseña no vamos a tener buenas predicciones
usuario_agrup = reviews_ny.groupby(by='Id_Usuario').count().sort_values(by='Rating',ascending=False)
usuario_agrup

,Id_Restaurant,Rating,Reseña,Timestamp
Id_Usuario,,,,
1.066545e+20,159,159,159,159
1.048192e+20,147,147,147,147
1.084387e+20,81,81,68,81
1.150828e+20,81,81,81,81
1.116543e+20,80,80,54,80
...,...,...,...,...
1.112619e+20,1,1,0,1
1.046370e+20,1,1,1,1
1.112619e+20,1,1,0,1


In [10]:
#Limitamos a los que tienen mas de 10 votos
usuario_agrup[usuario_agrup['Rating']>10]

,Id_Restaurant,Rating,Reseña,Timestamp
Id_Usuario,,,,
1.066545e+20,159,159,159,159
1.048192e+20,147,147,147,147
1.084387e+20,81,81,68,81
1.150828e+20,81,81,81,81
1.116543e+20,80,80,54,80
...,...,...,...,...
1.039415e+20,11,11,10,11
1.085041e+20,11,11,5,11
1.177329e+20,11,11,1,11


In [11]:
reviews_ny_filt = reviews_ny[reviews_ny['Id_Usuario'].isin(usuario_agrup[usuario_agrup['Rating']>10].index)]
reviews_ny_filt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46298 entries, 6360732 to 7484519
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id_Usuario     46298 non-null  object
 1   Id_Restaurant  46298 non-null  object
 2   Rating         46298 non-null  int64 
 3   Reseña         29034 non-null  object
 4   Timestamp      46298 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 2.1+ MB


In [13]:
#Crea un objeto Reader y Dataset de Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(reviews_ny_filt[["Id_Usuario", "Id_Restaurant", "Rating"]], reader)

In [15]:
#Optimizamos hiperparámetros. Primero vamos a hacerlo con el SVD.
parametros = {'n_factors':np.arange(10,1000),
             'n_epochs':np.arange(5,50),
             'lr_all':np.arange(0.001,0.05,0.001),
             'reg_all':np.arange(0.005,0.1,0.005)}

rs = RandomizedSearchCV(SVD,param_distributions=parametros,cv=3,n_iter=30,random_state =42)
rs.fit(data)

In [16]:
results_df = pd.DataFrame(rs.cv_results)
results_df.sort_values(by='mean_test_rmse').head()

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,...,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_factors,param_n_epochs,param_lr_all,param_reg_all
3,0.929293,0.923372,0.930834,0.927833,0.003217,1,0.693526,0.695500,0.701359,0.696795,...,2,6.588249,0.473332,0.154919,0.061835,"{'n_factors': 217, 'n_epochs': 35, 'lr_all': 0...",217,35,0.025,0.07
9,0.931655,0.924557,0.931663,0.929292,0.003348,2,0.702470,0.700787,0.704968,0.702742,...,4,18.421245,0.236951,0.167182,0.030063,"{'n_factors': 430, 'n_epochs': 41, 'lr_all': 0...",430,41,0.031,0.07
8,0.933032,0.929278,0.931602,0.931304,0.001547,3,0.695521,0.698676,0.698757,0.697651,...,3,4.075852,0.209161,0.133643,0.034558,"{'n_factors': 171, 'n_epochs': 27, 'lr_all': 0...",171,27,0.049,0.02
11,0.934785,0.927888,0.934928,0.932533,0.003286,4,0.709013,0.708857,0.710433,0.709434,...,8,16.899916,0.076365,0.145418,0.029774,"{'n_factors': 502, 'n_epochs': 32, 'lr_all': 0...",502,32,0.024,0.07
12,0.935808,0.928836,0.933166,0.932603,0.002874,5,0.707607,0.707278,0.708324,0.707736,...,7,13.972641,0.126799,0.122118,0.001130,"{'n_factors': 404, 'n_epochs': 34, 'lr_all': 0...",404,34,0.025,0.05


In [17]:
#Tomamos solo las columnas que nos interesan.
resultados = results_df[['param_n_factors','param_n_epochs','param_lr_all',
                         'param_reg_all','mean_test_rmse']].sort_values(by='mean_test_rmse')
resultados

,param_n_factors,param_n_epochs,param_lr_all,param_reg_all,mean_test_rmse
3,217,35,0.025,0.070,0.927833
9,430,41,0.031,0.070,0.929292
8,171,27,0.049,0.020,0.931304
11,502,32,0.024,0.070,0.932533
12,404,34,0.025,0.050,0.932603
17,751,21,0.043,0.095,0.934008
0,551,48,0.012,0.095,0.934979
10,203,30,0.014,0.010,0.935073
6,276,37,0.047,0.010,0.935154
24,338,8,0.026,0.090,0.935948


In [18]:
#Definimos función para limpiar los nombres de los campos.
def shorten_param(param_name):
    if "param_" in param_name:
        return param_name.rsplit("param_", 1)[1]
    return param_name

In [28]:
#Limpiamos los nombres de los campos
resultados = resultados.rename(shorten_param, axis=1)
resultados.reset_index(inplace=True,drop=True)
resultados

,n_factors,n_epochs,lr_all,reg_all,mean_test_rmse
0,217,35,0.025,0.070,0.927833
1,430,41,0.031,0.070,0.929292
2,171,27,0.049,0.020,0.931304
3,502,32,0.024,0.070,0.932533
4,404,34,0.025,0.050,0.932603
5,751,21,0.043,0.095,0.934008
6,551,48,0.012,0.095,0.934979
7,203,30,0.014,0.010,0.935073
8,276,37,0.047,0.010,0.935154
9,338,8,0.026,0.090,0.935948


In [29]:
#Ploteamos las dependencias de cada hiperparámetro
import plotly.express as px

fig = px.parallel_coordinates(
    resultados,
    color="mean_test_rmse",
    color_continuous_scale=px.colors.sequential.Viridis,
)
fig.show()

Vamos a probar ahora realizar el mismo proceso pero con el modelo KNNWithMeans. Luego elegiremos los mejores hiperparametros y compararemos los resultados de ambos.

In [31]:
#Optimizamos hiperparámetros del modelo KNNWithMeans
parametros = {'k':np.arange(10,100),'min_k':np.arange(5),'sim_options':{'name':['cosine','msd','pearson']}}

rs2 = RandomizedSearchCV(KNNWithMeans,param_distributions=parametros,n_iter=100,random_state =42)
rs2.fit(data)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Co

Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.

Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done co

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pears

Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computin

In [32]:
results_df2 = pd.DataFrame(rs2.cv_results)
results_df2.sort_values(by='mean_test_rmse').head()

,split0_test_rmse,split1_test_rmse,split2_test_rmse,split3_test_rmse,split4_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,...,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_k,param_min_k,param_sim_options
99,0.938214,0.949711,0.925394,0.918536,0.930166,0.932404,0.010768,1,0.676659,0.684681,...,0.00792,1,1.195151,0.148748,0.263525,0.079651,"{'k': 14, 'min_k': 4, 'sim_options': {'name': ...",14,4,"{'name': 'pearson', 'user_based': True}"
90,0.938214,0.949711,0.925394,0.918536,0.930166,0.932404,0.010768,2,0.676659,0.684681,...,0.00792,2,0.989917,0.012328,0.219108,0.053979,"{'k': 87, 'min_k': 4, 'sim_options': {'name': ...",87,4,"{'name': 'pearson', 'user_based': True}"
84,0.938214,0.949711,0.925394,0.918536,0.930166,0.932404,0.010768,3,0.676659,0.684681,...,0.00792,3,0.983391,0.006254,0.190706,0.006799,"{'k': 93, 'min_k': 4, 'sim_options': {'name': ...",93,4,"{'name': 'pearson', 'user_based': True}"
61,0.938214,0.949711,0.925394,0.918536,0.930166,0.932404,0.010768,4,0.676659,0.684681,...,0.00792,5,1.049452,0.089192,0.243237,0.060533,"{'k': 58, 'min_k': 4, 'sim_options': {'name': ...",58,4,"{'name': 'pearson', 'user_based': True}"
65,0.938214,0.949711,0.925394,0.918536,0.930166,0.932404,0.010768,5,0.676659,0.684681,...,0.00792,4,1.001976,0.045242,0.220974,0.054644,"{'k': 37, 'min_k': 4, 'sim_options': {'name': ...",37,4,"{'name': 'pearson', 'user_based': True}"


In [35]:
results_df2.columns

Index(['split0_test_rmse', 'split1_test_rmse', 'split2_test_rmse',
       'split3_test_rmse', 'split4_test_rmse', 'mean_test_rmse',
       'std_test_rmse', 'rank_test_rmse', 'split0_test_mae', 'split1_test_mae',
       'split2_test_mae', 'split3_test_mae', 'split4_test_mae',
       'mean_test_mae', 'std_test_mae', 'rank_test_mae', 'mean_fit_time',
       'std_fit_time', 'mean_test_time', 'std_test_time', 'params', 'param_k',
       'param_min_k', 'param_sim_options'],
      dtype='object')

In [36]:
#Tomamos solo las columnas que nos interesan.
resultados2 = results_df2[['param_k','param_min_k','param_sim_options',
                         'mean_test_rmse']].sort_values(by='mean_test_rmse')
resultados2

,param_k,param_min_k,param_sim_options,mean_test_rmse
99,14,4,"{'name': 'pearson', 'user_based': True}",0.932404
90,87,4,"{'name': 'pearson', 'user_based': True}",0.932404
84,93,4,"{'name': 'pearson', 'user_based': True}",0.932404
61,58,4,"{'name': 'pearson', 'user_based': True}",0.932404
65,37,4,"{'name': 'pearson', 'user_based': True}",0.932404
...,...,...,...,...
71,61,1,"{'name': 'msd', 'user_based': True}",1.008061
41,34,0,"{'name': 'msd', 'user_based': True}",1.008072
39,33,0,"{'name': 'msd', 'user_based': True}",1.008076
92,30,0,"{'name': 'msd', 'user_based': True}",1.008084


In [37]:
#Limpiamos los nombres de los campos
resultados2 = resultados2.rename(shorten_param, axis=1)
resultados2.reset_index(inplace=True,drop=True)
resultados2

,k,min_k,sim_options,mean_test_rmse
0,14,4,"{'name': 'pearson', 'user_based': True}",0.932404
1,87,4,"{'name': 'pearson', 'user_based': True}",0.932404
2,93,4,"{'name': 'pearson', 'user_based': True}",0.932404
3,58,4,"{'name': 'pearson', 'user_based': True}",0.932404
4,37,4,"{'name': 'pearson', 'user_based': True}",0.932404
...,...,...,...,...
95,61,1,"{'name': 'msd', 'user_based': True}",1.008061
96,34,0,"{'name': 'msd', 'user_based': True}",1.008072
97,33,0,"{'name': 'msd', 'user_based': True}",1.008076
98,30,0,"{'name': 'msd', 'user_based': True}",1.008084


In [38]:
#Ploteamos las dependencias de cada hiperparámetro
import plotly.express as px

fig = px.parallel_coordinates(
    resultados2,
    color="mean_test_rmse",
    color_continuous_scale=px.colors.sequential.Viridis,
)
fig.show()

Aca se pueden ver patrones mas claros donde convendría apuntar (min_k mayores a 2 y sim_options = pearson) y quizás volver a optimizar. Pero directamente vamos a comparar ambos modelos con los mejores hiperparámetros haciendo cross validation y nos quedaremos con el que mejor resultado tenga.

In [46]:
#Chequeamos los mejores hiperparámetros de SVD
rs.best_params['rmse']

{'n_factors': 217, 'n_epochs': 35, 'lr_all': 0.025, 'reg_all': 0.07}

In [49]:
#Hacemos cross validation de SVD
cv = KFold(random_state=25)
model = SVD(n_factors=217, n_epochs=35, lr_all=0.025, reg_all=0.07)

for trainset, testset in cv.split(data):
    model.fit(trainset)
    predictions = model.test(testset)
    accuracy.rmse(predictions)

RMSE: 0.9202
RMSE: 0.9238
RMSE: 0.9197
RMSE: 0.9242
RMSE: 0.9276


In [50]:
#Chequeamos los mejores hiperparámetros de KNNWithMeans
rs2.best_params['rmse']

{'k': 58, 'min_k': 4, 'sim_options': {'name': 'pearson', 'user_based': True}}

In [52]:
#Hacemos cross validation de KNNWithMeans
model2 = KNNWithMeans(k=58, min_k=4, sim_options={'name': 'pearson', 'user_based': True})

for trainset, testset in cv.split(data):
    model2.fit(trainset)
    predictions2 = model2.test(testset)
    accuracy.rmse(predictions2)

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9344
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9340
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9299
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9341
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9300


Los valores de SVD son ligeramente mejores. Por lo tanto, vamos a usar este algoritmo para predecir. Lo ultimo que vamos a realizar es compararlo con el modelo con los hiperparámetros que vienen por default.

In [53]:
#Dividimos los datos en conjuntos de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.25, random_state=24)

In [54]:
#Instanciamos modelo por default
model_def = SVD()

In [55]:
#Entrenamos ambos modelos
model_def.fit(trainset)
model.fit(trainset)

In [56]:
#Sacamos las predicciones
predicciones_def = model_def.test(testset)
predicciones = model.test(testset)

In [57]:
#Obtenemos los rsme
print('Default:')
accuracy.rmse(predicciones_def)
print('Optimizado:')
accuracy.rmse(predicciones)

Default:
RMSE: 0.9296
Optimizado:
RMSE: 0.9319


0.9318954687089492

No se observan grandes cambios entre el modelo optimizado y el que viene por defecto. Vamos a exportar entonces el optimizado ya que gracias a su mayor valor de learning rate tiende a converger más rápido.

In [60]:
#Importamos la librería joblib para llevar el modelo entrenado a la API
import joblib

joblib.dump(model, 'sistema_recomendacion_rest.joblib')

['sistema_recomendacion_rest.joblib']